<a href="https://colab.research.google.com/github/aknip/Autogen-AutoGPT/blob/main/AutoGen_Snake-Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogen Snake-Game

Notes:
- Code execution within Notebook fails: "It seems like the code execution failed because the environment in which the code is being run does not support the `curses` library. The `curses` library is used for creating text-based user interfaces and is typically used in a terminal or command-line environment.
"
- Source: https://github.com/abhilashi/ai-explorations/blob/main/ai_scene_writer.py

In [1]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string): ··········


In [ ]:
!pip install pyautogen==0.1.3 docker -q

## Setup




In [3]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager


config_list = [{'model': 'gpt-4', 'api_key': CREDS['OpenAI']['v1']['credential']}]

llm_config = {
    "seed": 44,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 1200,
}


# Define the common working directory for all agents
working_directory = "game_files"

# Initialize the Player agent, responsible for providing gameplay feedback
player = UserProxyAgent(
    name="Player",
    system_message="Player: Your role is to provide feedback on the gameplay. Collaborate with the Game Designer to ensure the game meets desired expectations.",
    code_execution_config={
        "work_dir": working_directory,
        "use_docker": False,
        "timeout": 120,
        "last_n_messages": 1,
    },
)

# Initialize the Game Designer agent, responsible for designing the game
game_designer = AssistantAgent(
    name="Game_Designer",
    llm_config=llm_config,
    system_message="Game Designer: Design the snake game, ensuring all details are documented in 'game_design.txt'. Collaborate with the Player to align the design with feedback and expectations."
)

# Initialize the Programmer agent, responsible for coding the game
programmer = AssistantAgent(
    name="Programmer",
    llm_config=llm_config,
    system_message="Programmer: Code the snake game and save it in the working directory. For code execution, collaborate with the Code Executor. If feedback is needed, consult the Game Tester."
)

# Initialize the Game Tester agent, responsible for playtesting the game
game_tester = UserProxyAgent(
    name="Game_Tester",
    system_message="Game Tester: Playtest the game and provide feedback on gameplay mechanics and user experience. Report any bugs or glitches. Collaborate with the Programmer for any necessary adjustments.",
    code_execution_config={
        "work_dir": working_directory,
        "use_docker": False,
        "timeout": 120,
        "last_n_messages": 3,
    },
    human_input_mode="ALWAYS",
)

# Initialize the Code Executor agent, responsible for executing the game code
code_executor = UserProxyAgent(
    name="Code_Executor",
    system_message="Code Executor: Execute the provided code from the Programmer in the designated environment. Report outcomes and potential issues. Ensure the code follows best practices and recommend enhancements to the Programmer.",
    code_execution_config={
        "work_dir": working_directory,
        "use_docker": False,
        "timeout": 120,
        "last_n_messages": 3,
    },
    human_input_mode="NEVER",
)

# Set up the group chat with all the agents
groupchat = GroupChat(
    agents=[player, game_tester, game_designer, programmer, code_executor],
    messages=[],
    max_round=150
)

# Create a manager for the group chat using the GPT-4 configuration
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)




# Start Chat

In [4]:
# Start the conversation with the Player's message
player.initiate_chat(
    manager,
    message="Let's design and implement a snake game. I aim for it to be entertaining and challenging."
)

Player (to chat_manager):

Let's design and implement a snake game. I aim for it to be entertaining and challenging.

--------------------------------------------------------------------------------
Game_Designer (to chat_manager):

Absolutely, let's get started. Here's a basic outline of the game:

Title: Snake Feast

Objective: The player controls a snake, guiding it to eat food that appears randomly on the screen. Each time the snake eats, it grows longer. The game ends when the snake hits the screen border or its own body.

Gameplay Mechanics:
1. The snake moves in a straight line and does not stop.
2. The player can control the direction of the snake's movement (up, down, left, right).
3. Food appears randomly on the screen.
4. The snake grows longer each time it eats food.
5. The game ends if the snake hits the screen border or its own body.

Scoring:
1. Each piece of food eaten is worth 10 points.
2. The score is displayed on the screen.

Levels:
1. The game starts at level 1.
2

KeyboardInterrupt: ignored